In [1]:
import os
import pandas as pd
import os
from joblib import Parallel, delayed
import pickle
import networkx as nx
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import imageio
import os
from joblib import Parallel, delayed


#import data
df = pd.read_csv("df_normalizedcleanfinalscaled.csv")



,Unnamed: 0,yearid,year_1b,year_2,id_1b,id_2,coninc_1b,coninc_2,race_1b,sexnow_1b,sexnow1_2,affrmact_norm1b,gunlaw_norm1b,partyid_norm1b,affrmact_norm2,gunlaw_norm2,partyid_norm2
0,1,20180001,2018,2020,1,810,NaN,36960.0,1,NaN,1,-1.000000,1,-0.666667,-1.000000,1.0,-1.000000
1,2,20180006,2018,2020,6,815,NaN,25200.0,1,NaN,2,0.333333,1,0.333333,-0.333333,1.0,0.333333
2,3,20180011,2018,2020,11,819,112160.0,94080.0,1,NaN,1,-1.000000,1,-1.000000,-1.000000,1.0,-1.000000
3,4,20180016,2018,2020,16,822,47317.5,15960.0,1,NaN,2,-0.333333,1,0.000000,-1.000000,1.0,0.000000
4,5,20180063,2018,2020,63,836,38555.0,NaN,2,NaN,2,0.333333,1,1.000000,1.000000,1.0,1.000000


In [ ]:


#load static graph from file
with open("fixed_graph.pkl", "rb") as f:
    G = pickle.load(f)

#verify structure
print(f"Loaded fixed graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")


In [2]:

#extracting columns for topics for 2018 and 2020
opinion_cols_1b = ["affrmact_norm1b", "gunlaw_norm1b", "partyid_norm1b"]
opinion_cols_2 = ["affrmact_norm2", "gunlaw_norm2", "partyid_norm2"]

#clean data from NA vals
df_clean = df.dropna(subset=opinion_cols_1b + opinion_cols_2)

#structure opinion matrices
A_initial = df_clean[opinion_cols_1b].values 
A_final = df_clean[opinion_cols_2].values   

#number of nodes/indivduals 
N = A_initial.shape[0]  
l = A_initial.shape[1]  #number of topics, l=3 

lambda_val = 0.5

In [3]:

lambda_val = 0.5  # probability decays

#probability matrix on all topics
prob_matrix = np.zeros((N, N))

for i in range(N):
    for j in range(i + 1, N):  #no duplicate calculations
        distance = np.linalg.norm(A_initial[i] - A_initial[j])**2  
        prob_matrix[i, j] = np.exp(-(1/(4*lambda_val))*distance)
        prob_matrix[j, i] = prob_matrix[i, j]  #symmertic matrix


In [5]:

#computing clustering coefficient and centrality measures
clustering_coeffs = nx.clustering(G)
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)

#converting centrality and clustering metrics to a frame for analysis
network_metrics = pd.DataFrame({
    "Node": list(G.nodes()),
    "Degree": [G.degree(n) for n in G.nodes()],
    "Clustering Coefficient": [clustering_coeffs[n] for n in G.nodes()],
    "Degree Centrality": [degree_centrality[n] for n in G.nodes()],
    "Betweenness Centrality": [betweenness_centrality[n] for n in G.nodes()]
})

#compute average clustering based on topic alignment
topic_similarity = {"affirmative_action": [], "gun_laws": [], "party_id": []}

for i in range(N):
    for j in range(i + 1, N):
        distance = np.linalg.norm(A_initial[i] - A_initial[j])**2
        probability = np.exp(- (1 /(4*lambda_val))*distance) #prob connected
        
        #separate contributions from each topic dimension
        for idx, topic in enumerate(["affirmative_action", "gun_laws", "party_id"]):
            topic_distance = (A_initial[i, idx] - A_initial[j, idx])**2
            topic_prob = np.exp(- (1 / (4*lambda_val)) * topic_distance)
            topic_similarity[topic].append(topic_prob)

#computing average connection probability per topic
average_topic_similarity = {topic: np.mean(values) for topic, values in topic_similarity.items()}



network_metrics.to_csv("[DIRECTORY]/network_metrics2playerP2first.csv", index=False)
df_topic_similarity = pd.DataFrame.from_dict(average_topic_similarity, orient="index", columns=["Avg Probability"])
df_topic_similarity.to_csv("DIRECTORY]/topic_similarity2playerP2first.csv", index=True)



Network Metrics (First 5 Rows):
   Node  Degree  Clustering Coefficient  Degree Centrality  \
0     0     111                0.571499           0.395018   
1     1     113                0.546618           0.402135   
2     2      95                0.529675           0.338078   
3     3     131                0.611979           0.466192   
4     4     108                0.571305           0.384342   

   Betweenness Centrality  
0                0.002397  
1                0.002460  
2                0.002282  
3                0.002083  
4                0.002122  

Average Connection Probability Per Topic:
                    Avg Probability
affirmative_action         0.731862
gun_laws                   0.670030
party_id                   0.719685


In [6]:

#covar/interdep.
topics2018 = opinion_cols_1b
topics2020 = opinion_cols_2
df_change = df_clean[topics2020].values - df_clean[topics2018].values
C_covar = np.cov(df_change, rowvar=False)
C_df = pd.DataFrame(C_covar, index=topics2018, columns=topics2018)
C_stochastic = C_df.div(C_df.sum(axis=1), axis=0).clip(lower=0)
C_stochastic = C_stochastic.div(C_stochastic.sum(axis=1), axis=0).values




In [7]:
#Player and simulation
class Player:
    def __init__(self, theta_low, theta_high, delta, beta, topic_index, agent_id, push_toward, l, kappa, gamma):
        self.theta_low = theta_low
        self.theta_high = theta_high
        self.delta = delta
        self.beta = beta
        self.topic_index = topic_index
        self.agent_id = agent_id
        self.push_toward = push_toward
        self.l = l
        self.kappa = kappa
        self.gamma = gamma

    def compute_w_link(self, A_updated):
        N = A_updated.shape[0]
        w_link_matrix = np.zeros((N, N))
        for i in range(N):
            for j in range(i + 1, N):
                distance = np.linalg.norm(A_updated[i] - A_updated[j])
                w_link_matrix[i, j] = np.exp(-distance)
                w_link_matrix[j, i] = w_link_matrix[i, j]
        return w_link_matrix

    def send_message(self, network, initial_node, message, C, opponent_message, A_initial):
        active_nodes = set()
        claimed_nodes = {}
        A_updated = np.copy(A_initial)

        if opponent_message is None:
            opponent_message = np.full(self.l, 0)

        alignment_self = self.beta * (1 - np.linalg.norm(A_initial[initial_node] - message) / (2 * np.sqrt(self.l)))
        alignment_opponent = self.beta * (1 - np.linalg.norm(A_initial[initial_node] - opponent_message) / (2 * np.sqrt(self.l)))

        if alignment_self > alignment_opponent and alignment_self > self.theta_low:
            w_link_matrix = self.compute_w_link(A_updated)
            neighbors = list(network.neighbors(initial_node))
            Z_i = np.sum(w_link_matrix[initial_node, neighbors])
            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[initial_node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
            A_updated[initial_node] = (1 - self.kappa - self.gamma) * A_updated[initial_node] + self.kappa * (C @ message) + self.gamma * neighbor_influence
            active_nodes.add(initial_node)
            claimed_nodes[initial_node] = self.agent_id

        while True:
            new_active_nodes = set()
            w_link_matrix = self.compute_w_link(A_updated)
            for node in active_nodes:
                for neighbor in network.neighbors(node):
                    if neighbor in claimed_nodes:
                        continue
                    if w_link_matrix[node, neighbor] > self.delta:
                        alignment_self = self.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(self.l)))
                        alignment_opponent = self.beta * (1 - np.linalg.norm(A_updated[neighbor] - opponent_message) / (2 * np.sqrt(self.l)))
                        if alignment_self > alignment_opponent and alignment_self > self.theta_low:
                            neighbors = list(network.neighbors(node))
                            Z_i = np.sum(w_link_matrix[node, neighbors])
                            neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                            A_updated[neighbor] = (1 - self.kappa - self.gamma) * A_updated[neighbor] + self.kappa * (C @ message) + self.gamma * neighbor_influence
                            new_active_nodes.add(neighbor)
                            claimed_nodes[neighbor] = self.agent_id
            if not new_active_nodes:
                break
            active_nodes.update(new_active_nodes)

        total_influence = self.push_toward * np.sum(A_updated[:, self.topic_index] - A_initial[:, self.topic_index])
        return total_influence, A_updated, claimed_nodes

    def optimize_strategy(self, network, C, opponent_message, A_initial):
        best_node, best_message, max_influence = None, None, -float('inf')
        base_vectors = [np.eye(self.l)[i] for i in range(self.l)] + [-np.eye(self.l)[i] for i in range(self.l)]
        sampled_messages = [v for v in base_vectors] + [np.random.uniform(-1, 1, self.l) for _ in range(3)]

        for initial_node in network.nodes:
            for message in sampled_messages:
                influence, _, _ = self.send_message(network, initial_node, message, C, opponent_message, A_initial)
                if influence > max_influence:
                    max_influence = influence
                    best_node = initial_node
                    best_message = message

        if best_node is None:
            best_node = np.random.choice(list(network.nodes))
            best_message = np.ones(self.l) if self.push_toward == 1 else -1 * np.ones(self.l)

        return best_node, best_message, max_influence

#diffusion
def simulate_diffusion(player, network, initial_node, message, C, A_initial):
    active_nodes_history = []
    A_updated = np.copy(A_initial)
    active_nodes = set([initial_node])
    claimed_nodes = {initial_node: player.agent_id}
    active_nodes_history.append(set(active_nodes))

    while True:
        new_active_nodes = set()
        w_link_matrix = player.compute_w_link(A_updated)
        for node in active_nodes:
            for neighbor in network.neighbors(node):
                if neighbor in claimed_nodes:
                    continue
                if w_link_matrix[node, neighbor] > player.delta:
                    alignment = player.beta * (1 - np.linalg.norm(A_updated[neighbor] - message) / (2 * np.sqrt(player.l)))
                    if alignment > player.theta_low:
                        neighbors = list(network.neighbors(node))
                        Z_i = np.sum(w_link_matrix[node, neighbors])
                        neighbor_influence = np.sum([A_updated[j] * w_link_matrix[node, j] for j in neighbors]) / Z_i if Z_i > 0 else 0
                        A_updated[neighbor] = (1 - player.kappa - player.gamma) * A_updated[neighbor] + player.kappa * (C @ message) + player.gamma * neighbor_influence
                        new_active_nodes.add(neighbor)
                        claimed_nodes[neighbor] = player.agent_id
        if not new_active_nodes:
            break
        active_nodes.update(new_active_nodes)
        active_nodes_history.append(set(active_nodes))

    return active_nodes_history, A_updated, claimed_nodes



In [9]:
#sim wrapper for parallel
def run_two_player_simulation(sim, topic_index, G, C, A_initial, player_config):
    player1 = Player(topic_index=topic_index, agent_id=1, push_toward=1, l=A_initial.shape[1], **player_config)
    player2 = Player(topic_index=topic_index, agent_id=2, push_toward=-1, l=A_initial.shape[1], **player_config)

    n2, m2, infl2 = player2.optimize_strategy(G, C, None, A_initial)
    n1, m1, infl1 = player1.optimize_strategy(G, C, m2, A_initial)

    hist2, A2, c2 = simulate_diffusion(player2, G, n2, m2, C, A_initial)
    hist1, A1, c1 = simulate_diffusion(player1, G, n1, m1, C, A_initial)

    #visualize_diffusion_as_gif(G, hist1, hist2, A1, A2, c1, c2, topic_index, m1, m2)

    return [
        {"Simulation": sim + 1, "Topic": topic_index, "Agent": 1, "Optimal Node": n1, "Total Influence": infl1, "Optimal Message": m1, "Nodes Influenced": len(c1)},
        {"Simulation": sim + 1, "Topic": topic_index, "Agent": 2, "Optimal Node": n2, "Total Influence": infl2, "Optimal Message": m2, "Nodes Influenced": len(c2)}
    ]

#parallel processing fro multi sim
def run_parallel_two_player_simulations(G, C, A_initial, num_simulations=5, topics=[0, 1, 2]):
    player_config = {
        "theta_low": 0.3,
        "theta_high": 0.7,
        "delta": 0.4,
        "beta": 1,
        "kappa": 0.3,
        "gamma": 0.2
    }

    jobs = [(sim, topic, G, C, A_initial, player_config) for sim in range(num_simulations) for topic in topics]

    results_nested = Parallel(n_jobs=-1)(
        delayed(run_two_player_simulation)(*job) for job in jobs
    )

    results = [entry for sublist in results_nested for entry in sublist]
    df_results = pd.DataFrame(results)
    df_results.to_csv("[DIRECTORY]/2player_parallel_simulation_resultsP2First.csv", index=False)
    print("Saved to 2player_parallel_simulation_resultsP2First.csv")
    return df_results

In [10]:
df_results=run_parallel_two_player_simulations(G, C_stochastic, A_initial, num_simulations=25, topics=[0, 1, 2])

Saved to 2player_parallel_simulation_resultsP2First.csv


,Simulation,Topic,Agent,Optimal Node,Total Influence,Optimal Message,Nodes Influenced
0,1,0,1,5,55.648636,"[-0.43608566246192426, 0.7003155536052332, 0.0...",122
1,1,0,2,267,10.184355,"[-1.0, -0.0, -0.0]",49
2,1,1,1,133,21.859016,"[0.0, 1.0, 0.0]",119
3,1,1,2,136,50.020952,"[-0.8735002992066436, -0.05946537212659386, -0...",176
4,1,2,1,142,30.765176,"[-0.6343225668061121, 0.8625902348449672, -0.1...",195
...,...,...,...,...,...,...,...
145,25,0,2,267,10.184355,"[-1.0, -0.0, -0.0]",49
146,25,1,1,133,21.859016,"[0.0, 1.0, 0.0]",119
147,25,1,2,102,79.200622,"[-0.17256377329433237, 0.27596634043078594, -0...",209
148,25,2,1,141,31.177965,"[0.0, 0.0, 1.0]",109
